In [1]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 1)
model.load_state_dict(torch.load('speed_model_1.pth'))

device = torch.device('cuda')
model = model.to(device).half().eval()

In [2]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

In [3]:
torch.save(model_trt.state_dict(), 'speed_model_trt_1.pth')

In [1]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('speed_model_trt_1.pth'))

In [1]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_width=1280, capture_height=720, capture_fps=5)

In [2]:
camera.running = True

In [3]:
import ipywidgets

In [4]:
speed_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.001)

In [6]:
speed_slider

FloatSlider(value=0.0, max=1.0, step=0.001)

In [7]:
from utils import preprocess

def execute(change):
    image = change['new'] // 2 + change['old'] // 2
    image_preview.value = bgr8_to_jpeg(image)
    data = preprocess(image).half()
    speed = float(model_trt(data)[0][0].cpu())
    speed_slider.value = speed

In [8]:
import ipywidgets

image_preview = ipywidgets.Image()

image_preview

Image(value=b'')

In [9]:
camera.observe(execute, names='value')

In [12]:
camera.unobserve_all()

In [65]:
image = imga // 2 + imgb // 2
image_preview.value = bgr8_to_jpeg(image)

In [11]:
imga = camera.value.copy()

In [12]:
imgb = camera.value.copy()

In [13]:
execute({'new': imga, 'old': imgb})